# Python101

Mentoring material for python basics

## Generators

In [1]:
def my_generator_func():
    yield 'start'
    for i in range(3):
        yield i
    yield 'end'
    return 'done'

for item in my_generator_func():
    print(item)

start
0
1
2
end


In [2]:
# retrieve all elements from a generator
list(my_generator_func())

['start', 0, 1, 2, 'end']

### Generator != Generator function

In [3]:
# the generator is stateful
my_generator = my_generator_func()

print('next:', next(my_generator))
print('next:', next(my_generator))
print('next:', next(my_generator))
print('remaining:', list(my_generator))
print('remaining:', list(my_generator))

next: start
next: 0
next: 1
remaining: [2, 'end']
remaining: []


In [4]:
# the generator function returns a new generator each time
print('new call:', list(my_generator_func()))
print('new call:', list(my_generator_func()))

new call: ['start', 0, 1, 2, 'end']
new call: ['start', 0, 1, 2, 'end']


### Generators can be inifinite!

In [5]:
# does not have to be finite
def new_counter():
    i = 0
    while True:
        yield i
        i += 1

counter1 = new_counter()
print('next1:', next(counter1))
print('next1:', next(counter1))
print('next1:', next(counter1))
print('next1:', next(counter1))

next1: 0
next1: 1
next1: 2
next1: 3


In [6]:
# already exists in stdlib's itertools!
# https://docs.python.org/3/library/itertools.html#itertools.count
import itertools
        
counter2 = itertools.count()

print('next2:', next(counter2))
print('next2:', next(counter2))
print('next2:', next(counter2))

next2: 0
next2: 1
next2: 2


In [7]:
# DON'T DO THIS THOUGH!
# list(counter2)

### Generators make lazy evaluation possible

In [8]:
def take(n, iterable):
    """
    Return first n items of the iterable as a list
    from https://docs.python.org/3/library/itertools.html#recipes
    """
    return list(itertools.islice(iterable, n))

counter3 = itertools.count()
print('take 5:', take(5, counter3))
print('take 8:', take(8, counter3))

take 5: [0, 1, 2, 3, 4]
take 8: [5, 6, 7, 8, 9, 10, 11, 12]


In [9]:
import time

def heavy_generator():
    for i in range(4):
        time.sleep(0.15)
        print('processing...', i)
        yield i

filtered = (x for x in heavy_generator() if x % 2)
print('filtered:', filtered)

squared = (x ** 2 for x in filtered)
print('squared:', squared)

evaluated = list(squared)  # not evaluated until here!
print('evaluated:', evaluated)

filtered: <generator object <genexpr> at 0x10f386f68>
squared: <generator object <genexpr> at 0x10f386d58>
processing... 0
processing... 1
processing... 2
processing... 3
evaluated: [1, 9]


In [10]:
def fibonacci():
    a, b = 1, 1
    while True:
        yield a
        a, b = a + b, a

# we only compute the terms we need
take(10, fibonacci())

[1, 2, 3, 5, 8, 13, 21, 34, 55, 89]

#### Exercises

In [11]:
# Implement a generator yielding 'A', 'B', 'C', 'A', 'B', ...

def abc_generator():
    yield None

take(5, abc_generator())  # should be ['A', 'B', 'C', 'A', 'B']

[None]

In [12]:
# Implement take without using itertools.islice

def my_take(n, iterator):
    pass

my_take(5, itertools.count())  # should be [0, 1, 2, 3, 4]

In [13]:
# Implement an infinte fizzbuzz

def fizzbuzz():
    yield None

expected = [
    1,
    2,
    'fizz',
    4,
    'buzz',
    'fizz',
    7,
    8,
    'fizz',
    'buzz',
    11,
    'fizz',
    13,
    14,
    'fizzbuzz',
]

take(15, fizzbuzz())  # should be == expected

[None]

## Iterators

Basically everything you can loop on:

In [14]:
# A string:
for c in 'Hello':
    print(c)

H
e
l
l
o


In [15]:
# A list:
for i in [5, 7, 1, 0]:
    print(i)

5
7
1
0


In [16]:
# A range:
for i in range(4):
    print(i)

0
1
2
3


In [17]:
# A tuple:
for i in (6, 8, 9):
    print(i)

6
8
9


In [18]:
# A set:
for i in {1, 2, 3, 4}:
    print(i)

1
2
3
4


In [19]:
# A generator:
for i in my_generator_func():
    print(i)

start
0
1
2
end


In [20]:
# but also real-world examples: a file, a database cursor...
i = 0
with open('/dev/random', 'rb') as f:
    for chunk in f:
        i += 1
        print(chunk[:5])
        if i > 2:
            break

b'\xd09\xbf\x02\xda'
b'\xb1y\xfd\x1c\xb8'
b'_\x87\x98\xf9\xfb'


> BTW, you never do this in python. Use `enumerate`!

In [21]:
# use enumerate(iterator) to get a new iterator with the index
with open('/dev/random', 'rb') as f:
    for i, chunk in enumerate(f):
        print(chunk[:5])
        if i > 2:
            break

b'\x9a\xe5\xe9\xbds'
b'\xe4\xe1\x8b7\xa8'
b')\xd4}\x90\xe3'
b'\x8d\x04\xeb\xc4S'


### Pro-tip: the list constructor

You can use the list constructor on any (non-infinite) iterator to see all its elements:

In [22]:
# a string:
list('Hello')

['H', 'e', 'l', 'l', 'o']

In [23]:
# a list:
list([5, 7, 1, 0])

[5, 7, 1, 0]

In [24]:
# a range:
list(range(4))

[0, 1, 2, 3]

In [25]:
# a tuple:
list((6, 8, 9))

[6, 8, 9]

In [26]:
# a set:
list({1, 2, 3, 4})

[1, 2, 3, 4]

In [27]:
# a generator:
list(my_generator_func())

['start', 0, 1, 2, 'end']

In [28]:
# an enumerate:
list(enumerate('python'))

[(0, 'p'), (1, 'y'), (2, 't'), (3, 'h'), (4, 'o'), (5, 'n')]

### Pro-tip: use constructors for casting!

You can use other constructors on any iterator (simple casting)

In [29]:
# a tuple:
tuple(range(5))

(0, 1, 2, 3, 4)

In [30]:
# a set:
set('SnoopDoggyDog')

{'D', 'S', 'g', 'n', 'o', 'p', 'y'}

In [31]:
# a dict (iterator of tuples):
dict([(1, 3), (4, 9)])

{1: 3, 4: 9}

In [32]:
# a dict (again:
dict(enumerate('hello'))

{0: 'h', 1: 'e', 2: 'l', 3: 'l', 4: 'o'}

### Pro-tip: Use list-comprehensions!

1. Less code, clearer code
2. Better performance!
3. They are idiomatic of good python code (i.e. they are "pythonic")

In [33]:
def next_ascii_char(c):
    """
    A simple function we'll use below to illustrate
    """
    return chr(ord(c) + 1)

In [34]:
# same as the "map" function
[next_ascii_char(c) for c in 'Hello']

['I', 'f', 'm', 'm', 'p']

In [35]:
# same as the "filter" function
[c for c in 'Hello' if c != 'l']

['H', 'e', 'o']

In [36]:
# filter & map at the same time
[next_ascii_char(c) for c in 'Hello' if c != 'l']

['I', 'f', 'p']

In [37]:
# two nested loops - not necessary a good idea ;)
emails = [f'{x}@{y}' for x in ['pierre', 'paul', 'jack'] for y in ['gmail.com', 'hotmail.com', 'yopmail.com']]
emails

['pierre@gmail.com',
 'pierre@hotmail.com',
 'pierre@yopmail.com',
 'paul@gmail.com',
 'paul@hotmail.com',
 'paul@yopmail.com',
 'jack@gmail.com',
 'jack@hotmail.com',
 'jack@yopmail.com']

> This kind of small loops are:
- less pythonic
- less performant
- more verbose
- less readable

### It's not just *list* comprehensions!

In [38]:
# dict comprehensions:
{ord(c): c for c in 'Hello'}

{72: 'H', 101: 'e', 108: 'l', 111: 'o'}

In [39]:
# dict comprehensions:
{ord(c): c for c in 'Hello'}

{72: 'H', 101: 'e', 108: 'l', 111: 'o'}

In [40]:
# set comprehensions:
{i for i in range(8) if i % 3}

{1, 2, 4, 5, 7}

In [41]:
# Generator comprehensions:
all_ints_with_one = (i for i in itertools.count() if '1' in str(i))

all_ints_with_one # not evaluated here

<generator object <genexpr> at 0x10f386eb8>

In [42]:
# evaluated lazily!
take(15, all_ints_with_one)  # replay this block

[1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 31, 41, 51]

### Anti-pattern: never do this for simple loops!

In [43]:
birds = ('pidget', 'eagle', 'falcon', 'pidget')  # some iterator

# ugly golang-style ;)
results = []
for bird in birds:
    results.append(bird.upper())
    
results

['PIDGET', 'EAGLE', 'FALCON', 'PIDGET']

In [44]:
# or even worse: ugly C-style ;)
results = set()
for i in range(len(birds)):
    results.add(birds[i].upper())
    
results

{'EAGLE', 'FALCON', 'PIDGET'}

### Another typical examples: welcome `str.join`!

- unlike JS, defined in the string and not the array
- makes sense because not restricted to arrays!
- works on any `Sequence[str]`

In [45]:
'-'.join('ABCDEF')  # a str is a sequence of chars that are themselves str

'A-B-C-D-E-F'

In [46]:
','.join(['First', 'Second', 'Third'])  # a list

'First,Second,Third'

In [47]:
''.join('You are tearing me appart Lisa'.split())  # another list

'YouaretearingmeappartLisa'

In [48]:
# even with a generator comprehension:
','.join(s for s in ['First', 'Second', 'Third'] if 'i' in s)

'First,Third'

### Pro-tip: A lot of functions take any iterator (not a list)

In [49]:
sorted('Awesome')

['A', 'e', 'e', 'm', 'o', 's', 'w']

In [50]:
list(reversed('Awesome'))

['e', 'm', 'o', 's', 'e', 'w', 'A']

In [51]:
# lazy iterator as well:
reversed('Awesome')

#### Exercises

In [52]:
# How many distinct characters in a phrase?
# (should ignore the case: 'A' and 'a' should be counted as the same char)

def distinct_chars(s):
    pass

distinct_chars('This Is A Really Long String') # should be 13

In [53]:
# Remove all words that are smaller than n characters
def remove_small_words(s, n):
    pass

remove_small_words('What the hell are you doing', 3) # should be 'What hell doing'

In [54]:
# Create a string in the shape 1-2-3-4...
def generate_int_list_string(n):
    pass

generate_int_list_string(5) # should be '1-2-3-4-5'

## Decorators

In [55]:
# just a function taking a function as argument (and usually returning a function)
def my_decorator(fn):
    def decorated_fn(*args, **kwargs):
        print('args:', args, 'kwargs:', kwargs)
        result = fn(*args, **kwargs)
        print('result:', result)
        return result
    return decorated_fn

In [56]:
# without synctactic sugar

def func1(x, y):
    print('x:', x, 'y:', y)
    return 2 * x + y

func1 = my_decorator(func1)

func1(55, y=1)

args: (55,) kwargs: {'y': 1}
x: 55 y: 1
result: 111


111

In [57]:
# with synctactic sugar (exactly the same!)

@my_decorator
def func2(x, y):
    print('x:', x, 'y:', y)
    return 2 * x + y

func2(55, y=1)

args: (55,) kwargs: {'y': 1}
x: 55 y: 1
result: 111


111

> Don't confuse the syncatic sugar and the concept

In [58]:
# but we have some issues:
print(func1.__name__)
print(func2.__name__)
print(func1)
print(func2)

decorated_fn
decorated_fn
<function my_decorator.<locals>.decorated_fn at 0x10f3b0b70>
<function my_decorator.<locals>.decorated_fn at 0x10f3dd0d0>


In [59]:
# we'll need some help!
from functools import wraps

# functools.wraps is a base util to create clean decorators!

In [60]:
# just a function taking a function as argument (and returning a function)
def my_decorator_wrapped(fn):
    @wraps(fn)
    def decorated_fn(*args, **kwargs):
        print('args:', args, 'kwargs:', kwargs)
        result = fn(*args, **kwargs)
        print('result:', result)
        return result
    return decorated_fn

@my_decorator_wrapped
def func3(x, y):
    """
    Solves all your problems.
    Doubles everything.
    """
    print('x:', x, 'y:', y)
    return 2 * x + y

# now we're fine!
print(func3.__name__)
print(func3)

func3
<function func3 at 0x10f3ddea0>


In [61]:
# also keeps the docstring untouched:
func3.__doc__

'\n    Solves all your problems.\n    Doubles everything.\n    '

In [62]:
# the behaviour is unchanged
func3(32, y=1)

args: (32,) kwargs: {'y': 1}
x: 32 y: 1
result: 65


65

### Decorators != Decorator factories

In [63]:
# a decorator: takes a function as parameter

@my_decorator_wrapped
def func4():
    pass

In [64]:
# a decorator factory: takes parameters and returns a decorator

def my_decorator_factory(name):
    def my_decorator(fn):
        @wraps(fn)
        def decorated_fn(*args, **kwargs):
            print('before:', name)
            result = fn(*args, **kwargs)
            print('after:', name)
            return result
        return decorated_fn
    return my_decorator

@my_decorator_factory('my-name')
def func5():
    print('fn')

func5()

before: my-name
fn
after: my-name


> `my_decorator_factory` is a decorator factory

> `my_decorator_factory('my-name')` is a decorator

> `my_decorator_factory('my-name')(some_function)` is a decorated function

### Decorators don't have to return functions!

In [65]:
# "fun" example (don't use this!):

def iife(fn):
    """
    JS-like IIFE (Immediately Invoked Function Expressions)
    """
    return fn()

@iife
def toto():
    print('preparing...')
    return 'toto'.upper()

toto

preparing...


'TOTO'

## Context managers

In [66]:
# typical use case: manage resources (files, db connections...)
# (think "defer" in golang, "finally" in JS/python)

def crash():
    print('> before')
    
    try:
        crash_because_not_defined()
    finally:
        print('> finally')  # will be called

    print('> after')  # will not be called

try:
    crash()
except NameError as err:
    print(err)

> before
> finally
name 'crash_because_not_defined' is not defined


In [67]:
# easily create your own manager with contextlib
from contextlib import contextmanager

# https://docs.python.org/3/library/contextlib.html#contextlib.contextmanager

In [68]:
@contextmanager
def my_context_manager(name):
    print('> enter')
    yield name.upper()
    print('> exit')
    
def crash():
    print('> before')
    with my_context_manager('managueur') as cm:
        print(f'> got cm: {cm}')
        crash_because_not_defined()

    print('> after')  # will not be called

try:
    crash()
except NameError as err:
    print(err)

> before
> enter
> got cm: MANAGUEUR
name 'crash_because_not_defined' is not defined


In [69]:
# can also be done with a class implementing two methods: __enter__ & __exit__

class MyContextManager:
    def __init__(self, name):
        self.name = name

    def __enter__(self):
        print(f'> enter {self.name}')
        return self.name.upper()

    def __exit__(self, *args):
        print(f'> exit {self.name}')


    
def crash():
    print('> before')
    
    with MyContextManager('the danger zone') as cm:
        print(f'> got cm: {cm}')
        crash_because_not_defined()

    print('> after')  # will not be called

try:
    crash()
except NameError as err:
    print(err)

> before
> enter the danger zone
> got cm: THE DANGER ZONE
> exit the danger zone
name 'crash_because_not_defined' is not defined


In [70]:
# real world example: open a file

with open('/tmp/python_rocks.txt', 'w') as my_file:
    for email in emails:
        print(email, file=my_file)


with open('/tmp/python_rocks.txt', 'r') as my_file:
    saved = my_file.read()

saved

'pierre@gmail.com\npierre@hotmail.com\npierre@yopmail.com\npaul@gmail.com\npaul@hotmail.com\npaul@yopmail.com\njack@gmail.com\njack@hotmail.com\njack@yopmail.com\n'

In [71]:
# the file is closed outside of the context manager (you shouldn't even access the variable...)
try:
    my_file.read()
except ValueError as err:
    print(err)

I/O operation on closed file.


In [72]:
# another example: welcome tempfile!
# extract from https://docs.python.org/3.6/library/tempfile.html#examples
import tempfile
import os

with tempfile.NamedTemporaryFile() as fp:
    fp.write(b'Hello world!')
    temp_path = fp.name
    print('path:', temp_path)
    file_exists = os.path.isfile(temp_path)
    print('file_exists:', file_exists)
    fp.seek(0)
    stored = fp.read()
    print('stored:', stored)

file_exists = os.path.isfile(temp_path)
print('file_exists:', file_exists)

path: /var/folders/zv/mdh8jytn257dtlj6qknszfnm0000gp/T/tmpeywufrwr
file_exists: True
stored: b'Hello world!'
file_exists: False


## Error handling

### Be specific

Always catch the error you expect, close to where you expect it

In [73]:
def get_nested_key_safe(d: dict):
    try:
        return d['nested']['key']
    except KeyError:
        return 'default_value'

[get_nested_key_safe(d) for d in [{'nested': {'key': 55}}, {'nested': {}}, {}]]

[55, 'default_value', 'default_value']

### Let it raise

Don't try/except something that is not supposed to happen, and you can't do anything intelligent about it

In [74]:
 # if this is always supposed to be present, don't try to handle the case it isn't
def get_nested_key(d: dict):
    return d['nested']['key']

try:
    get_nested_key({})  # raises a very clear KeyError('nested')
except Exception as err:
    print(repr(err))

KeyError('nested',)


In [75]:
# the same is true for dict.get, which is a hidden try/except
# it is not always relevant and often abused

def get_nested_key_bad(d: dict):
    return d.get('nested')('key')

try:
    get_nested_key_bad({})  # raises an impossible to exploit TypeError
except Exception as err:
    print(repr(err))

TypeError("'NoneType' object is not callable",)


### EAFP: Easier to Ask for Forgiveness than Permission

More on this in the awesome [Python Anti-Patterns book](https://docs.quantifiedcode.com/python-anti-patterns/readability/asking_for_permission_instead_of_forgiveness_when_working_with_files.html)

In [76]:
# bad non-idiomatic implementation

def bad_load_from_file(filename):
    if os.path.isfile(filename):
        with open(filename) as f:
            return f.read()
    else:
        print('fallbacking...')
        return 'SOME_DEFAULT'

bad_load_from_file('/tmp/doesnotexist.txt')

fallbacking...


'SOME_DEFAULT'

In [77]:
# EAFP implementation

def eafp_load_from_file(filename):
    try:
        with open(filename) as f:
            return f.read()
    except OSError:
        print('fallbacking...')
        return 'SOME_DEFAULT'

eafp_load_from_file('/tmp/doesnotexist.txt')

fallbacking...


'SOME_DEFAULT'

> Of course, not only a Python thing! This philosophy prevents race conditions!

### Custom error handling (e.g. for business logic)

In [78]:
# define your own error type inheriting from Exception:
class FooError(Exception):
    pass

def foo():
    raise FooError(500)

err = None
try:
    foo()
# catch regarding the exception types you expect:
except FooError as e:
    err = e

err

__main__.FooError(500)